In [23]:
# from tensorflow.keras import
import chess
from chess import Board, engine
import pandas as pd
import random
import numpy

In [79]:
from traceback import print_exc


def get_scores_valuation_from_engine(engine_path: str, board: Board, depth: int, color_flag: str = "white") -> float:
    try:
        with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
            result = engine.analyse(board, chess.engine.Limit(depth=depth))
            if color_flag == "white":
                score = result['score'].white().score(mate_score=10000)
                # score = result
            else:
                score = result['score'].black().score(mate_score=10000)
                # score = result
            return score
    except Exception:
        print_exc()


In [71]:
# this function will create our x (board)
def random_board(depth):
  board = chess.Board()
  for _ in range(depth):
    all_moves = list(board.legal_moves)
    random_move = random.choice(all_moves)
    print(random_move)
    board.push(random_move)
    if board.is_game_over():
      break
  return board
# board = random_board(14)
board = chess.Board()
# board.set_fen(fen='Q4b1r/ppp2ppp/4k3/4p3/3P4/8/PPP2PPP/RNBQKBNR b KQ - 0 8')
board.set_fen(fen='rnbqkbnr/ppppp2p/8/5pp1/4P3/8/PPPP1PPP/RNBQKBNR w KQkq g6 0 3')
print(board.board_fen())
print(board)

rnbqkbnr/ppppp2p/8/5pp1/4P3/8/PPPP1PPP/RNBQKBNR
r n b q k b n r
p p p p p . . p
. . . . . . . .
. . . . . p p .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


In [26]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [33]:
board3d = split_dims(board)

In [80]:
scores = get_scores_valuation_from_engine('engines/stockfish_15_x64_avx2.exe', board, 14, "white")
print(scores)

9999
